In [28]:
import pandas as pd
import os

# Define the paths to the horse and race data files
horse_data_files = [f"C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_{year}.csv" for year in range(1990, 2021)]
race_data_files = [f"C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_{year}.csv" for year in range(1990, 2021)]
forward_data_file = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward.csv"

horse_data_list = [pd.read_csv(file, low_memory=False) for file in horse_data_files]
race_data_list = [pd.read_csv(file, low_memory=False) for file in race_data_files]  
forward_data = pd.read_csv(forward_data_file)

# Concatenate all years' data into single DataFrames
horse_data = pd.concat(horse_data_list, ignore_index=True)
race_data = pd.concat(race_data_list, ignore_index=True)


In [22]:
# Check for missing values
print(horse_data.isnull().sum())
print(race_data.isnull().sum())
print(forward_data.isnull().sum())



rid                   0
horseName             0
age                 309
saddle           179168
decimalPrice          0
isFav                 0
trainerName        2850
jockeyName          125
position              0
positionL        783662
dist            1074385
weightSt              0
weightLb              0
overWeight      4012988
outHandicap     3994332
headGear        3119695
RPR              864877
TR              1681610
OR              1795263
father             1289
mother             4480
gfather          119374
runners               0
margin                0
weight                0
res_win               0
res_place             0
price           4012304
dtype: int64
rid                 0
course              0
time                0
date                0
title               1
rclass         152992
band           231309
ages            13054
distance            0
condition          48
hurdles        274953
prizes              0
winningTime         0
prize           19708
metric 

In [29]:

# Define a function to drop columns with more than a specified percentage of null values
def drop_high_null_columns(df, threshold=0.7):
    null_percentage = df.isnull().sum() / len(df)
    columns_to_drop = null_percentage[null_percentage > threshold].index
    return df.drop(columns=columns_to_drop)

# Drop columns with more than 70% null values
horse_data_cleaned = drop_high_null_columns(horse_data, threshold=0.7)
race_data_cleaned = drop_high_null_columns(race_data, threshold=0.7)
forward_data_cleaned = drop_high_null_columns(forward_data, threshold=0.7)



In [11]:
# Check for missing values
print(horse_data_cleaned.isnull().sum())
print(race_data_cleaned.isnull().sum())
print(forward_data_cleaned.isnull().sum())



rid                   0
horseName             0
age                 309
saddle           179168
decimalPrice          0
isFav                 0
trainerName        2850
jockeyName          125
position              0
positionL        783662
dist            1074385
weightSt              0
weightLb              0
RPR              864877
TR              1681610
OR              1795263
father             1289
mother             4480
gfather          119374
runners               0
margin                0
weight                0
res_win               0
res_place             0
dtype: int64
rid                 0
course              0
time                0
date                0
title               1
rclass         152992
band           231309
ages            13054
distance            0
condition          48
hurdles        274953
prizes              0
winningTime         0
prize           19708
metric              0
countryCode         0
ncond               0
class               0
dtype: int64
co

In [30]:
import pandas as pd
import re

# Define cleaning and normalization functions
def clean_distance(dist):
    if pd.isnull(dist):
        return None
    dist_str = str(dist).lower()
    miles_match = re.search(r'(\d+)m', dist_str)
    furlongs_match = re.search(r'(\d+)f', dist_str)
    miles = int(miles_match.group(1)) if miles_match else 0
    furlongs = 0
    if furlongs_match:
        furlongs = int(furlongs_match.group(1))
    fractional_match = re.search(r'(\d+)/(\d+)f', dist_str)
    if fractional_match:
        furlongs += int(fractional_match.group(1)) / int(fractional_match.group(2))
    return miles * 8 + furlongs

def normalize_time(time_str):
    if pd.isnull(time_str):
        return None
    time_parts = time_str.split(':')
    if len(time_parts) == 2:
        return int(time_parts[0]) * 60 + int(time_parts[1])
    elif len(time_parts) == 3:
        return int(time_parts[0]) * 3600 + int(time_parts[1]) * 60 + int(time_parts[2])
    return None

def clean_ages(ages):
    if pd.isnull(ages):
        return None
    match = re.search(r'\d+', str(ages))
    return int(match.group(0)) if match else None

def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9\s]', '', title).strip() if title else title


def extract_numeric(value):
    if pd.isnull(value):
        return None
    match = re.search(r'\d+', str(value))
    return int(match.group(0)) if match else None


def clean_band(band):
    if pd.isnull(band) or band.strip() == '':
        return None  # Return None for empty or missing values

    band = str(band).strip()  # Remove leading/trailing whitespace

    # Handle ranges like '80-109', split and return the average of the range
    if '-' in band:
        band_range = band.split('-')

        # Check if both parts of the range are valid numbers
        if len(band_range) == 2 and band_range[0].strip() and band_range[1].strip():
            try:
                lower_bound = int(band_range[0].strip())
                upper_bound = int(band_range[1].strip())
                return (lower_bound + upper_bound) / 2  # Return the average
            except ValueError:
                return None  # If any part can't be converted to int, return None

    # If no range, return the numeric value if valid
    if band.isnumeric():
        return int(band)

    return None  # Return None if the value is not valid

def clean_rclass(rclass):
    if pd.isnull(rclass):
        return None
    match = re.search(r'\d+', str(rclass))
    return int(match.group(0)) if match else None


# Drop 'prizes' column in race data
if 'prizes' in race_data_cleaned.columns:
    race_data_cleaned.drop('prizes', axis=1, inplace=True)
if 'positionL' in horse_data_cleaned.columns:
    horse_data_cleaned.drop('positionL', axis=1, inplace=True)


# Apply cleaning functions if the 'rclass' column exists
if 'rclass' in horse_data_cleaned.columns:
    horse_data_cleaned['rclass'] = horse_data_cleaned['rclass'].apply(clean_rclass)
if 'rclass' in forward_data_cleaned.columns:
    forward_data_cleaned['rclass'] = forward_data_cleaned['rclass'].apply(clean_rclass)

race_data_cleaned['distance'] = race_data_cleaned['distance'].apply(clean_distance)
if 'time' in race_data_cleaned.columns:
    race_data_cleaned['time'] = race_data_cleaned['time'].apply(normalize_time)
race_data_cleaned['ages'] = race_data_cleaned['ages'].apply(clean_ages)
race_data_cleaned['hurdles'] = race_data_cleaned['hurdles'].apply(extract_numeric)
race_data_cleaned['band'] = race_data_cleaned['band'].apply(clean_band)

# Print cleaned DataFrames
print(horse_data_cleaned.head(10))
print(race_data_cleaned.head(10))
print(forward_data_cleaned.head(10))


      rid         horseName  age  saddle  decimalPrice  isFav  \
0  271018        Combermere  6.0     0.0      0.222222      0   
1  271018     Royal Battery  6.0     0.0      0.090909      0   
2  271018           Just So  7.0     0.0      0.029412      0   
3  271018  Mandraki Shuffle  8.0     0.0      0.090909      0   
4  271018    Turnberry Dawn  8.0     0.0      0.047619      0   
5  271018         Yreka Bay  8.0     0.0      0.029412      0   
6  271018         Hi' Upham  8.0     0.0      0.058824      0   
7  271018    Lauderdale Lad  8.0     0.0      0.200000      0   
8  271018  Providence Lodge  8.0     0.0      0.125000      0   
9  271018     Duke Of Abson  5.0     0.0      0.038462      0   

       trainerName      jockeyName  position  dist  ...    TR  OR  \
0        R G Frost         J Frost         1   NaN  ...  94.0 NaN   
1       D H Barons         S Earle         2   NaN  ...  88.0 NaN   
2      J D Roberts      S Burrough         3  25.0  ...  71.0 NaN   
3  Olive

In [31]:
forward_data_cleaned['marketTime'] = pd.to_datetime(forward_data_cleaned['marketTime'], utc=True)
print("\nCleaned Forward Data:")
print(forward_data_cleaned)


Cleaned Forward Data:
              course  countryCode                marketTime  \
0           Limerick            0 2020-09-11 11:45:00+00:00   
1           Limerick            0 2020-09-11 11:45:00+00:00   
2           Limerick            0 2020-09-11 11:45:00+00:00   
3           Limerick            0 2020-09-11 11:45:00+00:00   
4           Limerick            0 2020-09-11 11:45:00+00:00   
...              ...          ...                       ...   
33703  Wolverhampton            0 2020-12-05 20:00:00+00:00   
33704  Wolverhampton            0 2020-12-05 20:00:00+00:00   
33705  Wolverhampton            0 2020-12-05 20:00:00+00:00   
33706  Wolverhampton            0 2020-12-05 20:00:00+00:00   
33707  Wolverhampton            0 2020-12-05 20:00:00+00:00   

                                                   title  runners  \
0      Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
1      Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
2      Irish 

In [32]:
forward_data_cleaned.dtypes

course                       object
countryCode                   int64
marketTime      datetime64[ns, UTC]
title                        object
runners                       int64
condition                    object
prize                       float64
rclass                      float64
horseName                    object
trainerName                  object
jockeyName                   object
RPRc                        float64
TRc                         float64
OR                          float64
weightSt                      int64
weightLb                      int64
age                           int64
decimalPrice                float64
dtype: object

In [35]:
import pandas as pd

# Load the datasets
forward_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward_cleaned.csv")
races_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_cleaned.csv",low_memory=False)
horses_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned.csv")

# Display the first few rows to understand the data
print(forward_data.head())
print(races_data.head())
print(horses_data.head())


     course  countryCode                 marketTime  \
0  Limerick            0  2020-09-11 11:45:00+00:00   
1  Limerick            0  2020-09-11 11:45:00+00:00   
2  Limerick            0  2020-09-11 11:45:00+00:00   
3  Limerick            0  2020-09-11 11:45:00+00:00   
4  Limerick            0  2020-09-11 11:45:00+00:00   

                                               title  runners  \
0  Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
1  Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
2  Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
3  Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   
4  Irish Stallion Farms EBF Fillies Maiden (Plus ...       14   

          condition    prize  rclass          horseName          trainerName  \
0  Yielding To Soft  13717.5     NaN  All Down To Rosie        Conor O'Dwyer   
1  Yielding To Soft  13717.5     NaN         Colfer Kay           K J Condon   
2  Yielding To Soft  13717.5     NaN  

In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load the datasets
forward_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward_cleaned.csv")
races_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_cleaned.csv", low_memory=False)
horses_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned.csv")

# Convert weight from stones and pounds to pounds
def convert_weight_to_pounds(row):
    # Assuming weightSt is in stones and weightLb is in pounds
    weight_in_pounds = (row['weightSt'] * 14) + row['weightLb']
    return weight_in_pounds

# Calculate average position and win ratio for horses
def calculate_performance_metrics(horses_df):
    horses_df['avg_position'] = horses_df.groupby('horseName')['position'].transform('mean')
    horses_df['win_ratio'] = horses_df.groupby('horseName')['position'].apply(lambda x: (x == 1).mean())
    horses_df['weight_in_pounds'] = horses_df.apply(convert_weight_to_pounds, axis=1)  # Apply weight conversion
    return horses_df

# Aggregate race features like race distance and total races per horse
def race_features(races_df, horses_df):
    # Merge race data with horse data
    race_horse_data = pd.merge(races_df, horses_df[['rid', 'horseName', 'avg_position', 'win_ratio', 'weight_in_pounds']], on='rid', how='left')
    
    # Aggregate total races and average distance per horse
    race_horse_data['total_races'] = race_horse_data.groupby('horseName')['rid'].transform('count')
    race_horse_data['avg_distance'] = race_horse_data.groupby('horseName')['distance'].transform('mean')
    return race_horse_data

# Handle marketTime feature from forward_data (example of a time-based feature)
def process_market_time(forward_df):
    # Convert marketTime to UTC (timezone-aware)
    forward_df['marketTime'] = pd.to_datetime(forward_df['marketTime'], utc=True)
    
    # Ensure 'now' is also timezone-aware in UTC
    current_time = pd.to_datetime('now', utc=True)
    
    # Calculate time to race (in seconds)
    forward_df['time_to_race'] = (forward_df['marketTime'] - current_time).dt.total_seconds()
    
    return forward_df

# Feature Engineering for forward_data
forward_data = process_market_time(forward_data)

# Feature Engineering for horses_data
horses_data = calculate_performance_metrics(horses_data)

# Merge forward_data with horses_data to include weight_in_pounds
forward_data = pd.merge(forward_data, horses_data[['horseName', 'weight_in_pounds']], on='horseName', how='left')

# Feature Engineering for races_data
race_horse_data = race_features(races_data, horses_data)
races_data['time_per_distance'] = races_data['winningTime'] / races_data['distance']

# Time-related features in forward_data
forward_data['day_of_week'] = forward_data['marketTime'].dt.dayofweek
forward_data['hour_of_day'] = forward_data['marketTime'].dt.hour
forward_data['minute_of_day'] = forward_data['marketTime'].dt.minute
forward_data['is_weekend'] = forward_data['marketTime'].dt.weekday >= 5


# Horse-specific features in forward_data (added age and weight-related features)
forward_data['horse_age_category'] = pd.cut(forward_data['age'], bins=[0, 3, 5, 10, 20], labels=["Young", "Adult", "Old", "Very Old"])
forward_data['weight_per_age'] = forward_data['weight_in_pounds'] / forward_data['age']


# Display the modified DataFrames
print(horses_data.head())
print(race_horse_data.head())
print(forward_data.head())


      rid         horseName  age  saddle  decimalPrice  isFav  \
0  271018        Combermere  6.0     0.0      0.222222      0   
1  271018     Royal Battery  6.0     0.0      0.090909      0   
2  271018           Just So  7.0     0.0      0.029412      0   
3  271018  Mandraki Shuffle  8.0     0.0      0.090909      0   
4  271018    Turnberry Dawn  8.0     0.0      0.047619      0   

       trainerName  jockeyName  position  dist  ...          mother  \
0        R G Frost     J Frost         1   NaN  ...  Queen's Parade   
1       D H Barons     S Earle         2   NaN  ...      All At Sea   
2      J D Roberts  S Burrough         3  25.0  ...    Just Camilla   
3  Oliver Sherwood  M Richards         4  45.0  ...      Indictment   
4      T B Hallett  P Richards         5  75.0  ...    Shuil Alainn   

          gfather  runners    margin  weight res_win res_place avg_position  \
0  Sovereign Path       14  1.521003      69     1.0       1.0     9.103448   
1    Man The Rail       

In [55]:
# Save cleaned data to CSV
horse_data_cleaned.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_fe.csv", index=False)
race_data_cleaned.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_fe.csv", index=False)
forward_data_cleaned.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward_fe.csv", index=False)


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load the datasets
forward_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward_cleaned.csv")
races_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_cleaned.csv", low_memory=False)
horses_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned.csv")

# Convert weight from stones and pounds to pounds
def convert_weight_to_pounds(row):
    weight_in_pounds = (row['weightSt'] * 14) + row['weightLb']
    return weight_in_pounds

# Calculate average position and win ratio for horses
def calculate_performance_metrics(horses_df):
    # Filter out rows where position is NaN or not a number
    horses_df = horses_df[pd.to_numeric(horses_df['position'], errors='coerce').notnull()]
    horses_df['position'] = horses_df['position'].astype(int)

    # Calculate average position
    horses_df['avg_position'] = horses_df.groupby('horseName')['position'].transform('mean')
    
      
    horses_df['weight_in_pounds'] = horses_df.apply(convert_weight_to_pounds, axis=1)  # Apply weight conversion
    return horses_df

# Aggregate race features like race distance and total races per horse
def race_features(races_df, horses_df):
    race_horse_data = pd.merge(races_df, horses_df[['rid', 'horseName', 'avg_position', 'weight_in_pounds']], on='rid', how='left')
    race_horse_data['total_races'] = race_horse_data.groupby('horseName')['rid'].transform('count')
    race_horse_data['avg_distance'] = race_horse_data.groupby('horseName')['distance'].transform('mean')
    return race_horse_data

# Handle marketTime feature from forward_data (example of a time-based feature)
def process_market_time(forward_df):
    forward_df['marketTime'] = pd.to_datetime(forward_df['marketTime'], utc=True)
    current_time = pd.to_datetime('now', utc=True)
    forward_df['time_to_race'] = (forward_df['marketTime'] - current_time).dt.total_seconds()
    return forward_df

# Feature Engineering for forward_data
forward_data = process_market_time(forward_data)

# Feature Engineering for horses_data
horses_data = calculate_performance_metrics(horses_data)

# Merge forward_data with horses_data to include weight_in_pounds
forward_data = pd.merge(forward_data, horses_data[['horseName', 'weight_in_pounds']], on='horseName', how='left')

# Feature Engineering for races_data
race_horse_data = race_features(races_data, horses_data)
races_data['time_per_distance'] = races_data['winningTime'] / races_data['distance']

# Time-related features in forward_data
forward_data['day_of_week'] = forward_data['marketTime'].dt.dayofweek
forward_data['hour_of_day'] = forward_data['marketTime'].dt.hour
forward_data['minute_of_day'] = forward_data['marketTime'].dt.minute
forward_data['is_weekend'] = forward_data['marketTime'].dt.weekday >= 5

# Horse-specific features in forward_data (added age and weight-related features)
forward_data['horse_age_category'] = pd.cut(forward_data['age'], bins=[0, 3, 5, 10, 20], labels=["Young", "Adult", "Old", "Very Old"], include_lowest=True)
forward_data['weight_per_age'] = forward_data['weight_in_pounds'] / forward_data['age']

# Save the DataFrames to CSV files
forward_data.to_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\forward_processed.csv", index=False)
horses_data.to_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\horses_processed.csv", index=False)
race_horse_data.to_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\race_horse_processed.csv", index=False)

# Display the modified DataFrames
print(horses_data.head())
print(race_horse_data.head())
print(forward_data.head())


      rid         horseName  age  saddle  decimalPrice  isFav  \
0  271018        Combermere  6.0     0.0      0.222222      0   
1  271018     Royal Battery  6.0     0.0      0.090909      0   
2  271018           Just So  7.0     0.0      0.029412      0   
3  271018  Mandraki Shuffle  8.0     0.0      0.090909      0   
4  271018    Turnberry Dawn  8.0     0.0      0.047619      0   

       trainerName  jockeyName  position  dist  ...       father  \
0        R G Frost     J Frost         1   NaN  ...      Absalom   
1       D H Barons     S Earle         2   NaN  ...  Norfolk Air   
2      J D Roberts  S Burrough         3  25.0  ...        Sousa   
3  Oliver Sherwood  M Richards         4  45.0  ...     Mandalus   
4      T B Hallett  P Richards         5  75.0  ...    Fair Turn   

           mother         gfather  runners    margin weight res_win res_place  \
0  Queen's Parade  Sovereign Path       14  1.521003     69     1.0       1.0   
1      All At Sea    Man The Rail     

In [4]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import IterativeImputer

# Load the datasets
forward_data = pd.read_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\forward_processed.csv",low_memory=False)
races_data = pd.read_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\horses_processed.csv",low_memory=False)
horses_data = pd.read_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\race_horse_processed.csv",low_memory=False)

# Remove rows with missing values in categorical columns (non-numeric)
forward_data = forward_data.dropna(subset=forward_data.select_dtypes(exclude=['float64', 'int64']).columns)
races_data = races_data.dropna(subset=races_data.select_dtypes(exclude=['float64', 'int64']).columns)
horses_data = horses_data.dropna(subset=horses_data.select_dtypes(exclude=['float64', 'int64']).columns)

# Initialize the IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Apply the imputer to fill missing values in numerical columns
forward_data_imputed = imputer.fit_transform(forward_data.select_dtypes(include=['float64', 'int64']))
races_data_imputed = imputer.fit_transform(races_data.select_dtypes(include=['float64', 'int64']))
horses_data_imputed = imputer.fit_transform(horses_data.select_dtypes(include=['float64', 'int64']))

# Convert the imputed arrays back to DataFrames
forward_data_imputed = pd.DataFrame(forward_data_imputed, columns=forward_data.select_dtypes(include=['float64', 'int64']).columns)
races_data_imputed = pd.DataFrame(races_data_imputed, columns=races_data.select_dtypes(include=['float64', 'int64']).columns)
horses_data_imputed = pd.DataFrame(horses_data_imputed, columns=horses_data.select_dtypes(include=['float64', 'int64']).columns)

# Merge the imputed numerical data with the original categorical columns
forward_data_imputed_full = pd.concat([forward_data.select_dtypes(exclude=['float64', 'int64']).reset_index(drop=True), forward_data_imputed], axis=1)
races_data_imputed_full = pd.concat([races_data.select_dtypes(exclude=['float64', 'int64']).reset_index(drop=True), races_data_imputed], axis=1)
horses_data_imputed_full = pd.concat([horses_data.select_dtypes(exclude=['float64', 'int64']).reset_index(drop=True), horses_data_imputed], axis=1)

# Check for any remaining missing values in the final DataFrames
print("Missing values in forward_data_imputed_full:\n", forward_data_imputed_full.isnull().sum())
print("Missing values in races_data_imputed_full:\n", races_data_imputed_full.isnull().sum())
print("Missing values in horses_data_imputed_full:\n", horses_data_imputed_full.isnull().sum())

# Save the imputed data to CSV
forward_data_imputed_full.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/forward_imputed.csv", index=False)
races_data_imputed_full.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/race_horse_imputed.csv", index=False)
horses_data_imputed_full.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_imputed.csv", index=False)

print("Missing values in categorical columns have been removed, and numerical values have been imputed. Both have been saved!")


Missing values in forward_data_imputed_full:
 course                0
marketTime            0
title                 0
condition             0
horseName             0
trainerName           0
jockeyName            0
is_weekend            0
horse_age_category    0
countryCode           0
runners               0
prize                 0
rclass                0
RPRc                  0
TRc                   0
OR                    0
weightSt              0
weightLb              0
age                   0
decimalPrice          0
time_to_race          0
weight_in_pounds      0
day_of_week           0
hour_of_day           0
minute_of_day         0
weight_per_age        0
dtype: int64
Missing values in races_data_imputed_full:
 horseName           0
trainerName         0
jockeyName          0
dist                0
father              0
mother              0
gfather             0
rid                 0
age                 0
saddle              0
decimalPrice        0
isFav               0
position 

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load your datasets
horses_data_imputed_full = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_imputed.csv",low_memory=False)
races_data = pd.read_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/race_horse_imputed.csv",low_memory=False)

# Define the function for encoding categorical variables
def encode_categorical_columns(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    le = LabelEncoder()

    # Apply LabelEncoder to each categorical column with top 20 categories
    for col in categorical_columns:
        # Get the top 20 categories based on frequency
        top_20_categories = df[col].value_counts().nlargest(20).index
        
        # Replace categories not in top 20 with 'Other'
        df[col] = df[col].apply(lambda x: x if x in top_20_categories else 'Other')

        # Encode the categories
        df[col] = le.fit_transform(df[col])

    return df

# Define the function for scaling numerical columns
def scale_numerical_columns(df):
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()

    # Apply StandardScaler to each numerical column
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

# Encode categorical columns (top 20 categories) and scale numerical columns
encoded_horses_data = encode_categorical_columns(horses_data_imputed_full)
encoded_races_data = encode_categorical_columns(races_data)

encoded_horses_data = scale_numerical_columns(encoded_horses_data)
encoded_races_data = scale_numerical_columns(encoded_races_data)

# Save the encoded and scaled datasets
encoded_horses_data.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/encoded_horses_data.csv", index=False)
encoded_races_data.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/encoded_races_data.csv", index=False)



In [3]:

import pandas as pd
#Save the encoded and scaled datasets
horses_data=pd.read_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\horses_imputed.csv",low_memory=False)
races_data=pd.read_csv(r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\race_horse_imputed.csv",low_memory=False)

# Now, merge the datasets on 'rid'
final_merged_data = pd.merge(horses_data,races_data, on='rid', how='inner')

# Save the final merged dataset
final_merged_data.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/final_merged_data.csv", index=False)

# Display the final merged data
print(final_merged_data.head())


  course      date                                             title   rclass  \
0  Epsom  90/06/06  Ever Ready Derby (Group 1) 3YO Colts And Fillies  Class 1   
1  Epsom  90/06/06  Ever Ready Derby (Group 1) 3YO Colts And Fillies  Class 1   
2  Epsom  90/06/06  Ever Ready Derby (Group 1) 3YO Colts And Fillies  Class 1   
3  Epsom  90/06/06  Ever Ready Derby (Group 1) 3YO Colts And Fillies  Class 1   
4  Epsom  90/06/06  Ever Ready Derby (Group 1) 3YO Colts And Fillies  Class 1   

  condition countryCode ncond     horseName_x       rid   time  ...    RPR  \
0      Good          GB     1  Quest For Fame  305728.0  225.0  ...  119.0   
1      Good          GB     1  Quest For Fame  305728.0  225.0  ...  115.0   
2      Good          GB     1  Quest For Fame  305728.0  225.0  ...  112.0   
3      Good          GB     1  Quest For Fame  305728.0  225.0  ...  112.0   
4      Good          GB     1  Quest For Fame  305728.0  225.0  ...  111.0   

      TR          OR  runners    margin  wei

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset in chunks
final_merged_data_path = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/final_merged_data.csv"
chunk_size = 50000  # Adjust the chunk size based on your system's memory

chunks = pd.read_csv(final_merged_data_path, chunksize=chunk_size)

def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization is: {end_mem:.2f} MB")
    print(f"Decreased by {(100 * (start_mem - end_mem) / start_mem):.1f}%")

    return df

reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]
final_merged_data = pd.concat(reduced_chunks, ignore_index=True)

# Save the reduced memory dataset
reduced_memory_data_path = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/final_merged_data_reduced_memory.csv"
final_merged_data.to_csv(reduced_memory_data_path, index=False)


C:\Users\Elakkiya\AppData\Local\Temp\ipykernel_13232\2303596474.py:44: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]


Memory usage of dataframe is 17.55 MB
Memory usage after optimization is: 8.54 MB
Decreased by 51.4%


C:\Users\Elakkiya\AppData\Local\Temp\ipykernel_13232\2303596474.py:44: DtypeWarning: Columns (6,25) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]


Memory usage of dataframe is 17.55 MB
Memory usage after optimization is: 8.87 MB
Decreased by 49.5%


C:\Users\Elakkiya\AppData\Local\Temp\ipykernel_13232\2303596474.py:44: DtypeWarning: Columns (6,25) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]


...

Memory usage of dataframe is 17.55 MB
Memory usage after optimization is: 8.54 MB
Decreased by 51.4%


C:\Users\Elakkiya\AppData\Local\Temp\ipykernel_13232\2303596474.py:44: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]


Memory usage of dataframe is 17.55 MB
Memory usage after optimization is: 8.54 MB
Decreased by 51.4%


C:\Users\Elakkiya\AppData\Local\Temp\ipykernel_13232\2303596474.py:44: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_chunks = [reduce_memory_usage(chunk) for chunk in chunks]


Memory usage of dataframe is 17.55 MB
Memory usage after optimization is: 8.54 MB
Decreased by 51.4%
Memory usage of dataframe is 9.25 MB
Memory usage after optimization is: 4.35 MB
Decreased by 53.0%
